<a href="https://colab.research.google.com/github/caiohsfar/sentiment-analysis/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
!pip install spacy --upgrade
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 5.9 MB 3.3 MB/s 
     |████████████████████████████████| 456 kB 51.2 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 10.1 MB 42.1 MB/s 
     |████████████████████████████████| 623 kB 57.6 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 13.6 MB 2.6 MB/s 
  Attempting uninstall: en-core-web-sm
    Fou

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
import pandas as pd 
import string 
import random 
import os
import seaborn as sns
import numpy as np

In [ ]:
nlp = spacy.load("en_core_web_sm")
first_step_path = "./first_step_twitter.pkl"
text_col = "clean_text"
csv = pd.read_csv("twitter.csv")
csv.tail()

,clean_text,category
4262,rahul thinks that the economic conditions impr...,1
4263,modi never said ill give people 15lakhs stop b...,-1
4264,here fact check when modi govt comes power the...,1
4265,even dont agree doubt capable post but compari...,1
4266,what about fake promises lakh employment women...,-1


In [ ]:
import re
def preprocessing(text: str) -> str:  
    text = str(text).lower().strip()
    document = nlp(text)
    sanatized_words = []
    for token in document:
        #Get Lemmatization
        #Remove stop words, digits and ents
        if is_word_ok(token.text, STOP_WORDS):
            word = token.lemma_
            sanatized_words.append(word)
    return ' '.join(sanatized_words).strip()

def is_word_ok(word, stop_words):
    return word != "" and word.isalnum() and word not in stop_words and word[0] not in punctuation and not word.isdigit() and not bool(re.search(r'\d', word))


if not os.path.isfile(first_step_path):
    csv[text_col] = csv[text_col].apply(preprocessing)
    csv.to_pickle(first_step_path)


csv.tail()


,clean_text,category
4262,rahul think economic condition improve modi ru...,1
4263,modi say ill people stop bullshit proof talk,-1
4264,fact check modi govt come power huge decline p...,1
4265,nt agree doubt capable post compare efficiency...,1
4266,fake promise lakh employment women safety frau...,-1


In [ ]:
sanatized_df = pd.read_pickle(first_step_path)
df_word_count = (sanatized_df['clean_text'].str.split(' ', expand=True)
                         .apply(pd.value_counts, axis=1)
                         .fillna(0)  
                         .astype(int))




In [ ]:
df_word_count['clean_text'] = sanatized_df['clean_text'].to_numpy()
df_word_count['category'] = sanatized_df['category'].to_numpy()

df_word_count.to_pickle("./word_col_twitter.pkl")

df_word_count



,,aadesh,aadhaar,aadhar,aaega,aagaye,aage,aai,aaiye,aaj,aak,aaka,aakash,aake,aakhri,aal,aam,aankhon,aap,aapa,aapjds,aapka,aapke,aapki,aapko,aaropo,aate,aaya,aayega,abandon,abb,abbreviation,abcd,abd,abdicate,abdication,abduct,abduction,abdul,abdullah,...,youtube,youtuber,youyour,yr,yrother,yrrr,yrs,yrsshame,ysr,ysrcp,yup,yuva,yuvanestam,yuvraj,zada,zafar,zail,zakir,zalel,zamane,zardar,zardari,zarurat,zayed,zealand,zealanders,zee,zeenews,zero,ziddi,zindabad,zirakpur,zombie,zon,zone,zoom,zuckerberg,तरफ,नऊआ,clean_text
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,modi promise minimum government maximum govern...
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,talk nonsense continue drama vote modi
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,vote modi welcome bjp tell rahul main campaign...
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ask supporter prefix chowkidar name modi great...
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,answer powerful world leader today trump putin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,rahul think economic condition improve modi ru...
4263,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,modi say ill people stop bullshit proof talk
4264,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,fact check modi govt come power huge decline p...
4265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nt agree doubt capable post compare efficiency...


In [ ]:
def find_texts_where_exist_word(word):
  print("Texts that contains ", word)
  try:
    for i in df_word_count.index:
        if df_word_count.at[i, word] > 0:
          print("Text: " ,df_word_count.at[i, "clean_text"])
          print("Category: " , df_word_count.at[i, "category"])
  except KeyError as e:
        print("this word not exists in any text")
    

find_texts_where_exist_word("bad")



Texts that contains  bad
Text:  hindustan see bad hindu maj hindu rashtra thrash rascal face anti indian politiciansantinational urban naxal wait watch modis win pakistan mein bhi hindu hona garv baat hogi
Category:  1
Text:  malnutrition india constituency bad affect mallikarjun kharge gulbarga fare bad malnutrition indicator highprofile constituency thiruvananthapuram well
Category:  -1
Text:  steal give nirav modi shame recover personal loss cheat customer fraud bank open account thief pnb bad bank india shame
Category:  -1
Text:  modi govt come power late sanjay jhaji admit accept fact year upa govt bad people lose job
Category:  -1
Text:  tag fast grow youtuber care abt country thatacc modi destroy democracyhe blind fold politician look good bad bad good sarcasm
Category:  -1
Text:  language chowkidar tweet bhi sab bjp problem support modi amit sha nt use bad laugh culture rss
Category:  -1
Text:  sir request bjp candidate contest mandya feel bad vote time year vote mandya huge mo